<a href="https://www.kaggle.com/code/eliyahdeeb/assessment-2-interface-with-gradio?scriptVersionId=291632265" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Step 1: Install Gradio

In [1]:
!pip install gradio scikit-learn numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 45.7 MB/s eta 0:00:00
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.41.5
    Uninstalling pydantic_core-2.41.5:
      Successfully uninstalled pydantic_core-2.41.5
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.12.5
    Uninstalling pydantic-2.12.5:
      Successfully uninstalled pydantic-2.12.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.27 requires SQLAlchemy<3,>=1.4, but you have sqlalchemy 1.2.19 which is incompatible.


# Step 2: Import Libraries & Rider Data

Imports necessary libraries (gradio, numpy, NearestNeighbors).
- Defines the rider_data list (all forms with stats, images, quotes if added).
- Set FALLBACK_IMAGE for when no match or error.
- This block loads everything into memory — run it first after installs.

In [2]:
import gradio as gr
import numpy as np
from sklearn.neighbors import NearestNeighbors

# Rider stats: floats, lore-accurate official base values (punch = strength, kick = durability, 
# inverted 100m run time = speed, jump height/10 = tactics). 
# ONLY base/primary form for each Rider – no alternates or final/super upgrades.
# All stat tuples 100% unique. No sum limit.
# Images: your provided valid direct links (cleaned, standing still poses)
rider_data = [
    {"name": "Kuuga Mighty Form (Base)", "strength": 3.0, "speed": 5.2, "durability": 10.0, "tactics": 1.5, "image": "https://static.wikia.nocookie.net/kamenrider/images/f/f8/KRKu-Kuugamighty.png"},
    {"name": "Agito Ground Form (Base)", "strength": 7.0, "speed": 5.0, "durability": 15.0, "tactics": 3.0, "image": "https://static.wikia.nocookie.net/kamenrider/images/5/5d/KRAg-Agitoground.png"},
    {"name": "Ryuki (Base)", "strength": 5.0, "speed": 6.0, "durability": 4.0, "tactics": 5.0, "image": "https://static.wikia.nocookie.net/kamenrider/images/d/d4/KRRy-Ryuki_%28With_Dragclaw%29.png"},
    {"name": "Faiz (Base)", "strength": 4.0, "speed": 8.0, "durability": 3.0, "tactics": 5.0, "image": "https://static.wikia.nocookie.net/kamenrider/images/e/e1/KR555-Faiz.png"},
    {"name": "Blade (Base)", "strength": 2.8, "speed": 5.7, "durability": 4.8, "tactics": 3.3, "image": "https://static.wikia.nocookie.net/kamenrider/images/e/eb/KRBl-Blade.png"},
    {"name": "Hibiki (Base)", "strength": 9.0, "speed": 2.0, "durability": 6.0, "tactics": 3.0, "image": "https://static.wikia.nocookie.net/kamenrider/images/2/26/KRHi-Hibiki.png"},
    {"name": "Kabuto Rider Form (Base)", "strength": 3.0, "speed": 5.8, "durability": 7.0, "tactics": 3.7, "image": "https://static.wikia.nocookie.net/kamenrider/images/d/d7/KRKa-Kabutorider.png"},
    {"name": "Den-O Sword Form (Base)", "strength": 5.0, "speed": 5.2, "durability": 7.0, "tactics": 3.5, "image": "https://static.wikia.nocookie.net/kamenrider/images/e/ec/KRDCD-Decadeden-o.png"},
    {"name": "Kiva Kiva Form (Base)", "strength": 7.0, "speed": 5.0, "durability": 5.0, "tactics": 3.0, "image": "https://static.wikia.nocookie.net/kamenrider/images/c/c2/KRKi-Kiva_%28Future%29.png"},
    {"name": "Decade (Base)", "strength": 5.0, "speed": 5.0, "durability": 5.0, "tactics": 5.0, "image": "https://static.wikia.nocookie.net/kamenrider/images/9/9b/KRDCD-Decade.png"},
    {"name": "W CycloneJoker (Base)", "strength": 2.5, "speed": 4.5, "durability": 6.0, "tactics": 6.0, "image": "https://static.wikia.nocookie.net/kamenrider/images/e/e5/KRW-Doublecyclonejoker.png"},
    {"name": "OOO TaToBa Combo (Base)", "strength": 4.5, "speed": 5.5, "durability": 12.0, "tactics": 1.0, "image": "https://static.wikia.nocookie.net/kamenrider/images/6/67/KRO-OOOtatoba.png"},
    {"name": "Fourze Base States", "strength": 2.1, "speed": 6.2, "durability": 6.3, "tactics": 2.0, "image": "https://static.wikia.nocookie.net/kamenrider/images/f/fe/KRFo-Fourzebase.png"},
    {"name": "Wizard Flame Style (Base)", "strength": 4.2, "speed": 5.0, "durability": 7.3, "tactics": 3.5, "image": "https://static.wikia.nocookie.net/kamenrider/images/d/d8/KRWi-Wizardflame.png"},
    {"name": "Gaim Orange Arms (Base)", "strength": 6.7, "speed": 5.9, "durability": 10.2, "tactics": 2.8, "image": "https://static.wikia.nocookie.net/kamenrider/images/6/6b/KRGa-Gaimorange.png"},
    {"name": "Drive Type Speed (Base)", "strength": 6.0, "speed": 5.7, "durability": 10.4, "tactics": 3.2, "image": "https://static.wikia.nocookie.net/kamenrider/images/f/f6/KRDr-Drivespeed.png"},
    {"name": "Ghost Ore Damashii (Base)", "strength": 5.0, "speed": 5.8, "durability": 10.0, "tactics": 4.2, "image": "https://static.wikia.nocookie.net/kamenrider/images/6/66/KRGh-Ghostore.png"},
    {"name": "Ex-Aid Level 2 (Base)", "strength": 4.0, "speed": 5.0, "durability": 5.0, "tactics": 6.0, "image": "https://static.wikia.nocookie.net/kamenrider/images/0/03/KREA-Exaidl2.png"},
    {"name": "Build RabbitTank (Base)", "strength": 2.2, "speed": 5.1, "durability": 5.8, "tactics": 4.5, "image": "https://static.wikia.nocookie.net/kamenrider/images/d/d6/KRBu-Buildrabbittank.png"},
    {"name": "Zi-O (Base)", "strength": 6.0, "speed": 5.0, "durability": 5.0, "tactics": 4.0, "image": "https://static.wikia.nocookie.net/kamenrider/images/0/0a/KRZiO-Zi-O.png/revision/latest/scale-to-width-down/339?cb=20180902020826"}
]

FALLBACK_IMAGE = "https://static.wikia.nocookie.net/kamenrider/images/9/9e/Kamen_Rider_logo.png"

# Step 3: Define the "Henshin" Function

Defines the core function that:
- Calculates total points from sliders.
- Clamps/rounds if over 20 (optional — remove if you want no limit).
- If exactly 20 → uses Nearest Neighbors to find the closest form.
- Returns result text (form name, comparison, quote, hint) and image.
- If not 20 → returns adjustment message and fallback image.

In [3]:
def henshin(strength, speed, durability, tactics):
    # Clamp sliders to valid 0–10 range
    strength = max(0, min(10, strength))
    speed = max(0, min(10, speed))
    durability = max(0, min(10, durability))
    tactics = max(0, min(10, tactics))
    
    # NEW CHECK: If any stat is exactly 0, show error message
    if strength == 0 or speed == 0 or durability == 0 or tactics == 0:
        text = "Please enter a value greater than 0 for all stats (Strength, Speed, Durability, Tactics)."
        image = FALLBACK_IMAGE
        return text, image
    
    # Normal processing (user point as array)
    user_stats = np.array([[strength, speed, durability, tactics]])
    
    # Rider stats array
    rider_stats = np.array([[r["strength"], r["speed"], r["durability"], r["tactics"]] for r in rider_data])
    
    # Nearest Neighbors model
    nn = NearestNeighbors(n_neighbors=1, metric='euclidean')
    nn.fit(rider_stats)
    _, idx = nn.kneighbors(user_stats)
    closest = rider_data[idx[0][0]]
    
    # Safe image handling
    image = FALLBACK_IMAGE
    try:
        if closest["image"]:
            image = closest["image"]
    except:
        pass
    
    # Result text
    text = f"Closest Rider Match: {closest['name']}\n\nYour Stats:\nStrength: {strength}\nSpeed: {speed}\nDurability: {durability}\nTactics: {tactics}\n\n(No point limit — always shows the nearest form)"
    
    return text, image

# Step 4: Create and Launch the Gradio Interface

Sets up the visual app:
- Applies theme (red/purple/slate, Orbitron font).
- Adds title, instructions, sliders (0–10, step 0.1).
- Creates "Henshin!" submit button.
- Adds output boxes for text result and rider image.
- Adds reset button.
- Connects button clicks to henshin() and reset.

In [4]:
theme = gr.themes.Default(
    primary_hue="red",
    secondary_hue="purple",
    neutral_hue="slate",
    font=[gr.themes.GoogleFont("Orbitron"), "sans-serif"]
)

with gr.Blocks(theme=theme) as demo:
    gr.Markdown("# Heisei Rider Form Matcher")
    gr.Markdown("""
**No point limit** — enter any values (0–10) and get the closest Heisei Rider form based on your build.
Each form reflects its lore strengths — power, speed, resilience, or strategy.

**How to use:**
- Adjust sliders freely (0–10, 0.1 precision)
- Click 'Submit' to see the closest matching Rider
- Any total works — the system always finds the nearest form
- Reset to start fresh

Discover your Rider.
    """)
    
    with gr.Row():
        strength = gr.Slider(0, 10, step=0.1, value=0.0, label="Strength")
        speed_sldr = gr.Slider(0, 10, step=0.1, value=0.0, label="Speed")
        durability = gr.Slider(0, 10, step=0.1, value=0.0, label="Durability")
        tactics = gr.Slider(0, 10, step=0.1, value=0.0, label="Tactics")
    
    submit_btn = gr.Button("Submit!", variant="primary")
    
    text_out = gr.Textbox(label="Result", lines=10)
    img_out = gr.Image(label="Rider Form", height=500)
    
    reset_btn = gr.Button("Reset")

    inputs = [strength, speed_sldr, durability, tactics]
    outputs = [text_out, img_out]

    submit_btn.click(henshin, inputs=inputs, outputs=outputs)
    
    reset_btn.click(lambda: (0.0, 0.0, 0.0, 0.0), outputs=inputs)

demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://280a578c748c11fa95.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
